# Zonal Stats

This UDF shows how to extract zonal statistics from a raster for areas that correspond to a set of vector boundaries.

In [18]:
import fused


@fused.udf
def udf(
    bbox, min_zoom=15, table="s3://fused-asset/infra/building_msft_us/", chip_len=256
):

    # Load utility functions
    table_to_tile = fused.load(
        "https://github.com/fusedio/udfs/tree/f928ee1/public/common/"
    ).utils.table_to_tile
    geom_stats = fused.load(
        "https://github.com/fusedio/udfs/tree/f928ee1/public/common/"
    ).utils.geom_stats
    dsm_to_tile = fused.load(
        "https://github.com/fusedio/udfs/tree/f928ee1/public/DSM_JAXA_Example"
    ).utils.dsm_to_tile

    # Conditionally run the UDF if zoom level is high enough, to avoid blowing up the memory
    if bbox.z[0] >= min_zoom:

        # Load GeoDataFrame (Vector)
        gdf = table_to_tile(bbox, table, min_zoom)

        # Load DSM array (Raster)
        arr = dsm_to_tile(bbox, z_levels=[4, 6, 9, 11], verbose=False)

        # Run zonal statistics
        gdf_zonal = geom_stats(gdf, arr, chip_len=chip_len)
        return gdf_zonal
    else:
        print("Please zoom more... (US Only)")


arr = fused.run(udf=udf, x=5239, y=12664, z=15, engine="realtime")
arr

,geometry,stats,count,fused_index
787,"POLYGON ((-122.43259 37.78323, -122.43246 37.7...",42.666667,105,0
790,"POLYGON ((-122.43368 37.78044, -122.43358 37.7...",39.104046,173,1
816,"POLYGON ((-122.43817 37.78252, -122.43814 37.7...",47.529412,34,2
824,"POLYGON ((-122.43981 37.78232, -122.43979 37.7...",45.666667,54,3
1584,"POLYGON ((-122.43277 37.78417, -122.43266 37.7...",37.753695,203,4
...,...,...,...,...
42219,"POLYGON ((-122.43795 37.78136, -122.43796 37.7...",52.000000,9,491
42222,"POLYGON ((-122.43833 37.78663, -122.43832 37.7...",47.000000,4,492
42223,"POLYGON ((-122.43843 37.78440, -122.43841 37.7...",54.000000,14,493
42230,"POLYGON ((-122.44004 37.78030, -122.44012 37.7...",58.000000,6,494
